In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from xgboost import XGBRegressor
from lightgbm import LGBMRegressor
from sklearn.experimental import enable_hist_gradient_boosting
from sklearn.ensemble import HistGradientBoostingRegressor
import sklearn
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_absolute_error
from sklearn.decomposition import PCA
import warnings
warnings.filterwarnings('ignore')

In [2]:
np.datetime64(np.datetime64('1991-10-01 00:00:00'),'D')

numpy.datetime64('1991-10-01')

In [3]:
# import training dataset

X_train = pd.read_csv('../data/dengue_features_train.csv')
y_train = pd.read_csv('../data/dengue_labels_train.csv')

In [4]:
X_train.head()

,city,year,weekofyear,week_start_date,ndvi_ne,ndvi_nw,ndvi_se,ndvi_sw,precipitation_amt_mm,reanalysis_air_temp_k,...,reanalysis_precip_amt_kg_per_m2,reanalysis_relative_humidity_percent,reanalysis_sat_precip_amt_mm,reanalysis_specific_humidity_g_per_kg,reanalysis_tdtr_k,station_avg_temp_c,station_diur_temp_rng_c,station_max_temp_c,station_min_temp_c,station_precip_mm
0,sj,1990,18,1990-04-30,0.122600,0.103725,0.198483,0.177617,12.42,297.572857,...,32.00,73.365714,12.42,14.012857,2.628571,25.442857,6.900000,29.4,20.0,16.0
1,sj,1990,19,1990-05-07,0.169900,0.142175,0.162357,0.155486,22.82,298.211429,...,17.94,77.368571,22.82,15.372857,2.371429,26.714286,6.371429,31.7,22.2,8.6
2,sj,1990,20,1990-05-14,0.032250,0.172967,0.157200,0.170843,34.54,298.781429,...,26.10,82.052857,34.54,16.848571,2.300000,26.714286,6.485714,32.2,22.8,41.4
3,sj,1990,21,1990-05-21,0.128633,0.245067,0.227557,0.235886,15.36,298.987143,...,13.90,80.337143,15.36,16.672857,2.428571,27.471429,6.771429,33.3,23.3,4.0
4,sj,1990,22,1990-05-28,0.196200,0.262200,0.251200,0.247340,7.52,299.518571,...,12.20,80.460000,7.52,17.210000,3.014286,28.942857,9.371429,35.0,23.9,5.8


In [5]:
y_train.head()

,city,year,weekofyear,total_cases
0,sj,1990,18,4
1,sj,1990,19,5
2,sj,1990,20,4
3,sj,1990,21,3
4,sj,1990,22,6


In [6]:
X_train['week_start_date'] = pd.to_datetime(X_train['week_start_date'])

In [7]:
week_test = [np.datetime64(x,'D') for x in X_train['week_start_date'].tolist()]

In [8]:
def preparing_dataset(X,y,column_list,weeks,time=False, split =False):
    '''
    I will write an abstract of this function :)
    -----
    Input:
        X (pd.DataFrame): X(input) dataset
        y (pd.DataFrame): y(label) dataset
        column_list (list): a list of columns you want to keep
        weeks (int):
            1: get environmental features one week ago
            2: get environmental features two weeks ago
            ...
        time (Boolean): (if a value is True) add 'total_cases' column of two weeks ago 
        split (Boolean): (if a value is True) split the dataset by cities ('sj' or 'iq')
    -----
    Output:
        (if the value of split is True):
            df_X_train_concat (pd.DataFrame): 
            df_y_train_concat (pd.DataFrame): 
            df_X_val_concat (pd.DataFrame): 
            df_y_val_concat (pd.DataFrame): 
            
        (else):
            df_X_train_sj (pd.DataFrame): 
            df_y_train_sj (pd.DataFrame): 
            df_X_val_sj (pd.DataFrame): 
            df_y_val_sj (pd.DataFrame): 
            df_X_train_iq (pd.DataFrame): 
            df_y_train_iq (pd.DataFrame): 
            df_X_val_iq (pd.DataFrame): 
            df_y_val_iq (pd.DataFrame):    
    '''
    
    # make a copy of the X dataset
    df_X_train = X.copy()
    
   
    # fill missing values using pd.DataFrame interpolate 
    df_X_train = df_X_train.fillna(method = 'ffill')
    
    # change datatype of 'week_start_date' from string to np.datetime64
    df_X_train['week_start_date'] = pd.to_datetime(df_X_train['week_start_date'])
    
    # split the dataset by the city. 
    df_X_train_sj = df_X_train.loc[df_X_train['city']=='sj']
    df_X_train_iq = df_X_train.loc[df_X_train['city']=='iq']
    
    df_y_train_sj = y.loc[y['city']=='sj']
    df_y_train_iq = y.loc[y['city']=='iq']
    

    # this part is for transform the datasets to get environmental factors of three weeks ago 
    df_X_train_sj_time = df_X_train_sj.iloc[weeks:,:4].reset_index().iloc[:,1:]
    df_X_train_iq_time = df_X_train_iq.iloc[weeks:,:4].reset_index().iloc[:,1:]
    
    
    df_X_train_sj_features = df_X_train_sj.iloc[:-1*weeks,4:].reset_index().iloc[:,1:]
    df_X_train_iq_features = df_X_train_iq.iloc[:-1*weeks,4:].reset_index().iloc[:,1:]
    
    
    df_X_train_sj = pd.concat([df_X_train_sj_time,df_X_train_sj_features], axis=1)
    df_X_train_iq = pd.concat([df_X_train_iq_time,df_X_train_iq_features], axis=1) 
    
    df_X_train_sj =  df_X_train_sj.merge(y, left_on=['city','year','weekofyear'], right_on = ['city','year','weekofyear'], how= 'left')
    df_X_train_iq =  df_X_train_iq.merge(y, left_on=['city','year','weekofyear'], right_on = ['city','year','weekofyear'], how='left')
    
    # here I assigned 0 to city = sj and 1 to city = iq 
    df_X_train_sj['city'] = 0
    df_X_train_iq['city'] = 1    
    target_list = list(column_list)  
    
    # here, I make a new column: total_cases_two_weeks_ago. Please check below function (get_data_two_weeks_ago)
    if time==True:
        for i in range(1,3):
            print('week'+str(i))
            df_X_train_sj = get_data_weeks_ago(df_X_train_sj,i)
            df_X_train_iq = get_data_weeks_ago(df_X_train_iq,i)
            target_list.append('total_cases_'+str(i)+'_week_ago')
        
    # split train, validate dataset
    df_X_train_sj, df_X_val_sj = train_test_split(df_X_train_sj,train_size=0.7,shuffle=False)
    df_y_train_sj = df_X_train_sj['total_cases']
    df_X_train_sj = df_X_train_sj[target_list]
    df_y_val_sj = df_X_val_sj['total_cases']
    df_X_val_sj = df_X_val_sj[target_list]
    
    # split train, validate dataset

    df_X_train_iq, df_X_val_iq = train_test_split(df_X_train_iq,train_size=0.7,shuffle=False)
    df_y_train_iq = df_X_train_iq['total_cases']
    df_X_train_iq = df_X_train_iq[target_list]
    df_y_val_iq = df_X_val_iq['total_cases']
    df_X_val_iq = df_X_val_iq[target_list]
    
    # if split==False merge datasets about the two cities
    if split == False:
        df_X_train_concat = pd.concat([df_X_train_sj,df_X_train_iq],axis=0, ignore_index=True)
        df_y_train_concat = pd.concat([df_y_train_sj,df_y_train_iq],axis=0, ignore_index=True)
        df_X_val_concat = pd.concat([df_X_val_sj,df_X_val_iq],axis=0, ignore_index=True)
        df_y_val_concat = pd.concat([df_y_val_sj,df_y_val_iq],axis=0, ignore_index=True)
        return df_X_train_concat, df_y_train_concat, df_X_val_concat, df_y_val_concat
    
    else:
        return df_X_train_sj, df_y_train_sj, df_X_val_sj, df_y_val_sj,\
               df_X_train_iq, df_y_train_iq, df_X_val_iq, df_y_val_iq 

In [9]:
def get_data_weeks_ago(df,week):
    df_copy = df.copy()
    df_target = df.copy()
    column_name = str(week)+'_week_later'
    df_copy[column_name] = pd.to_datetime(df_copy['week_start_date'].apply(lambda x: test(x,week)))
    df_copy = df_copy[[column_name,'total_cases']]
    column_name_ = '_'+str(week)+'_week_ago'
    print([w for w in df_copy[column_name].tolist() if w not in df_target['week_start_date'].tolist()])
    df_target = df_target.merge(df_copy,left_on = 'week_start_date' , right_on = column_name, suffixes = ('',column_name_))
    return df_target

In [10]:
def test(x, week):
    t = x+np.timedelta64(7*week,'D')
    t = np.datetime64(t,'D')
    if  t in week_test:
        return t
    else:
        d = 0
        while t not in week_test:
            t = t+np.timedelta64(1,'D')
            d = d+1
            if d > 10:
                print(t)
                break
        return t

## 3. Forecasting with the previous value

In [11]:
# reset imported dataset

X_train = pd.read_csv('../data/dengue_features_train.csv')
y_train = pd.read_csv('../data/dengue_labels_train.csv')

In [12]:
feature_list_mosquito = ['city',
                         'year',
                         'weekofyear',
                        'ndvi_sw','reanalysis_dew_point_temp_k', 'reanalysis_specific_humidity_g_per_kg', 'reanalysis_min_air_temp_k',
               'station_min_temp_c',  'station_max_temp_c']

In [13]:
X_train.loc[X_train['city']=='sj'][feature_list_mosquito].head(10)

,city,year,weekofyear,ndvi_sw,reanalysis_dew_point_temp_k,reanalysis_specific_humidity_g_per_kg,reanalysis_min_air_temp_k,station_min_temp_c,station_max_temp_c
0,sj,1990,18,0.177617,292.414286,14.012857,295.9,20.0,29.4
1,sj,1990,19,0.155486,293.951429,15.372857,296.4,22.2,31.7
2,sj,1990,20,0.170843,295.434286,16.848571,297.3,22.8,32.2
3,sj,1990,21,0.235886,295.310000,16.672857,297.0,23.3,33.3
4,sj,1990,22,0.247340,295.821429,17.210000,297.5,23.9,35.0
5,sj,1990,23,0.181743,295.851429,17.212857,298.1,23.9,34.4
6,sj,1990,24,0.210271,295.865714,17.234286,297.7,23.3,32.2
7,sj,1990,25,0.133029,296.531429,17.977143,298.4,22.8,33.9
8,sj,1990,26,0.123600,296.378571,17.790000,297.7,22.8,33.9
9,sj,1990,27,0.202567,296.651429,18.071429,298.7,24.4,33.9


In [14]:
y_train.loc[y_train['city']=='sj'].head(10)

,city,year,weekofyear,total_cases
0,sj,1990,18,4
1,sj,1990,19,5
2,sj,1990,20,4
3,sj,1990,21,3
4,sj,1990,22,6
5,sj,1990,23,2
6,sj,1990,24,4
7,sj,1990,25,5
8,sj,1990,26,10
9,sj,1990,27,6


In [15]:
X_train.loc[X_train['city']=='iq'][feature_list_mosquito].head(10)

,city,year,weekofyear,ndvi_sw,reanalysis_dew_point_temp_k,reanalysis_specific_humidity_g_per_kg,reanalysis_min_air_temp_k,station_min_temp_c,station_max_temp_c
936,iq,2000,26,0.247200,295.184286,16.651429,293.1,20.7,32.5
937,iq,2000,27,0.241657,295.358571,16.862857,291.1,20.8,34.0
938,iq,2000,28,0.128014,295.622857,17.120000,292.6,20.7,33.0
939,iq,2000,29,0.200314,292.797143,14.431429,288.6,14.7,31.5
940,iq,2000,30,0.361043,293.957143,15.444286,291.5,19.1,33.3
941,iq,2000,31,0.255314,291.725714,13.421429,288.5,17.0,32.0
942,iq,2000,32,0.387271,293.821429,15.311429,292.1,19.9,34.0
943,iq,2000,33,0.382750,293.992857,15.465714,290.6,20.5,33.0
944,iq,2000,34,0.302714,292.904286,14.444286,291.3,19.0,34.0
945,iq,2000,35,0.324257,293.472857,15.057143,292.9,20.0,34.0


In [16]:
y_train.loc[y_train['city']=='sj'].head(20)

,city,year,weekofyear,total_cases
0,sj,1990,18,4
1,sj,1990,19,5
2,sj,1990,20,4
3,sj,1990,21,3
4,sj,1990,22,6
5,sj,1990,23,2
6,sj,1990,24,4
7,sj,1990,25,5
8,sj,1990,26,10
9,sj,1990,27,6


In [17]:
X_train, y_train, X_val, y_val = preparing_dataset(X_train, y_train, feature_list_mosquito , 3, time=True, split=False) 

week1
[Timestamp('2008-04-29 00:00:00')]
2010-07-13
[Timestamp('2010-07-13 00:00:00')]
week2
[Timestamp('2008-04-29 00:00:00'), Timestamp('2008-05-06 00:00:00')]
2010-07-13
2010-07-20
[Timestamp('2010-07-13 00:00:00'), Timestamp('2010-07-20 00:00:00')]


In [18]:
X_train.loc[X_train['city']==0].head(10)

,city,year,weekofyear,ndvi_sw,reanalysis_dew_point_temp_k,reanalysis_specific_humidity_g_per_kg,reanalysis_min_air_temp_k,station_min_temp_c,station_max_temp_c,total_cases_1_week_ago,total_cases_2_week_ago
0,0,1990,24,0.235886,295.310000,16.672857,297.0,23.3,33.3,2,6
1,0,1990,25,0.247340,295.821429,17.210000,297.5,23.9,35.0,4,2
2,0,1990,26,0.181743,295.851429,17.212857,298.1,23.9,34.4,5,4
3,0,1990,27,0.210271,295.865714,17.234286,297.7,23.3,32.2,10,5
4,0,1990,28,0.133029,296.531429,17.977143,298.4,22.8,33.9,6,10
5,0,1990,29,0.123600,296.378571,17.790000,297.7,22.8,33.9,8,6
6,0,1990,30,0.202567,296.651429,18.071429,298.7,24.4,33.9,2,8
7,0,1990,31,0.152929,296.041429,17.418571,298.0,21.7,31.7,6,2
8,0,1990,32,0.280667,296.334286,17.737143,297.3,23.9,32.8,17,6
9,0,1990,33,0.214557,295.960000,17.341429,297.1,22.8,31.7,23,17


In [19]:
X_val.loc[X_val['city']==1].tail(10)

,city,year,weekofyear,ndvi_sw,reanalysis_dew_point_temp_k,reanalysis_specific_humidity_g_per_kg,reanalysis_min_air_temp_k,station_min_temp_c,station_max_temp_c,total_cases_1_week_ago,total_cases_2_week_ago
424,1,2010,16,0.145243,296.568571,18.214286,294.2,21.0,35.3,6,9
425,1,2010,17,0.260114,298.161429,20.078571,294.6,22.5,35.4,8,6
426,1,2010,18,0.135886,297.002857,18.677143,294.3,21.2,33.0,4,8
427,1,2010,19,0.244400,297.678571,19.448571,294.7,22.5,33.5,2,4
428,1,2010,20,0.307943,296.468571,18.068571,294.6,21.4,36.2,7,2
429,1,2010,21,0.340286,296.975714,18.602857,294.6,21.4,33.2,6,7
430,1,2010,22,0.145600,296.738571,18.391429,294.0,21.2,33.7,5,6
431,1,2010,23,0.244500,293.227143,14.908571,290.1,20.0,34.0,8,5
432,1,2010,24,0.292514,296.825714,18.485714,294.5,22.4,35.4,1,8
433,1,2010,25,0.225657,296.452857,18.070000,291.9,21.7,34.7,1,1


In [20]:
# this is very complex. I merge X,y again and split the dataset by cities
X_val = pd.concat([X_val,y_val], axis=1)
X_val_sj = X_val.loc[X_val['city']==0]
X_val_iq = X_val.loc[X_val['city']==1]

In [21]:
y_val_sj = X_val_sj['total_cases']
X_val_sj = X_val_sj.drop('total_cases', axis=1)
y_val_iq = X_val_iq['total_cases']
X_val_iq = X_val_iq.drop('total_cases', axis=1)

In [22]:
for i in range(1,3):    
    X_val_sj['total_cases_'+str(i)+'_week_ago'].iloc[i:] = 0 

for i in range(1,3):    
    X_val_iq['total_cases_'+str(i)+'_week_ago'].iloc[i:] = 0 

In [23]:
X_train = X_train.drop(['year','weekofyear'], axis=1)

In [24]:
X = pd.concat([X_train, y_train], axis=1)

In [25]:
X_sj = X.loc[X['city']==0]
X_train_sj = X_sj.drop(['total_cases','city'], axis=1)
y_train_sj = X_sj['total_cases']

In [26]:
X_iq = X.loc[X['city']==1]
X_train_iq = X_iq.drop(['total_cases','city'], axis=1)
y_train_iq = X_iq['total_cases']

## XGBoost

### SJ

In [27]:
from sklearn.model_selection import GridSearchCV
from xgboost import XGBRegressor
xgb_regressor = XGBRegressor()
xgb_params = {'max_depth' : (3,4),
              'learning_rate': (0.05,0.06),
              'n_estimators' : (50,55,60,65),
             "min_child_weight" : [ 1, 2, 3],
             "gamma"            : [0.0,0.1, 0.2],
             "colsample_bytree" : [ 0.7,0.8,0.9] }

grid_  = GridSearchCV(xgb_regressor, xgb_params,cv=5, verbose=20) 
grid_.fit(X_train_sj,y_train_sj)

Fitting 5 folds for each of 432 candidates, totalling 2160 fits
[CV] colsample_bytree=0.7, gamma=0.0, learning_rate=0.05, max_depth=3, min_child_weight=1, n_estimators=50 
[CV]  colsample_bytree=0.7, gamma=0.0, learning_rate=0.05, max_depth=3, min_child_weight=1, n_estimators=50, score=0.817, total=   0.1s
[CV] colsample_bytree=0.7, gamma=0.0, learning_rate=0.05, max_depth=3, min_child_weight=1, n_estimators=50 
[CV]  colsample_bytree=0.7, gamma=0.0, learning_rate=0.05, max_depth=3, min_child_weight=1, n_estimators=50, score=0.697, total=   0.1s
[CV] colsample_bytree=0.7, gamma=0.0, learning_rate=0.05, max_depth=3, min_child_weight=1, n_estimators=50 
[CV]  colsample_bytree=0.7, gamma=0.0, learning_rate=0.05, max_depth=3, min_child_weight=1, n_estimators=50, score=0.746, total=   0.1s

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    0.0s remaining:    0.0s



[CV] colsample_bytree=0.7, gamma=0.0, learning_rate=0.05, max_depth=3, min_child_weight=1, n_estimators=50 
[CV]  colsample_bytree=0.7, gamma=0.0, learning_rate=0.05, max_depth=3, min_child_weight=1, n_estimators=50, score=0.816, total=   0.0s
[CV] colsample_bytree=0.7, gamma=0.0, learning_rate=0.05, max_depth=3, min_child_weight=1, n_estimators=50 
[CV]  colsample_bytree=0.7, gamma=0.0, learning_rate=0.05, max_depth=3, min_child_weight=1, n_estimators=50, score=0.840, total=   0.0s
[CV] colsample_bytree=0.7, gamma=0.0, learning_rate=0.05, max_depth=3, min_child_weight=1, n_estimators=55 
[CV]  colsample_bytree=0.7, gamma=0.0, learning_rate=0.05, max_depth=3, min_child_weight=1, n_estimators=55, score=0.808, total=   0.1s
[CV] colsample_bytree=0.7, gamma=0.0, learning_rate=0.05, max_depth=3, min_child_weight=1, n_estimators=55 


[Parallel(n_jobs=1)]: Done   3 out of   3 | elapsed:    0.1s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   4 out of   4 | elapsed:    0.1s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   5 out of   5 | elapsed:    0.2s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   6 out of   6 | elapsed:    0.3s remaining:    0.0s


[CV]  colsample_bytree=0.7, gamma=0.0, learning_rate=0.05, max_depth=3, min_child_weight=1, n_estimators=55, score=0.718, total=   0.1s
[CV] colsample_bytree=0.7, gamma=0.0, learning_rate=0.05, max_depth=3, min_child_weight=1, n_estimators=55 
[CV]  colsample_bytree=0.7, gamma=0.0, learning_rate=0.05, max_depth=3, min_child_weight=1, n_estimators=55, score=0.746, total=   0.0s
[CV] colsample_bytree=0.7, gamma=0.0, learning_rate=0.05, max_depth=3, min_child_weight=1, n_estimators=55 
[CV]  colsample_bytree=0.7, gamma=0.0, learning_rate=0.05, max_depth=3, min_child_weight=1, n_estimators=55, score=0.832, total=   0.1s
[CV] colsample_bytree=0.7, gamma=0.0, learning_rate=0.05, max_depth=3, min_child_weight=1, n_estimators=55 
[CV]  colsample_bytree=0.7, gamma=0.0, learning_rate=0.05, max_depth=3, min_child_weight=1, n_estimators=55, score=0.842, total=   0.1s
[CV] colsample_bytree=0.7, gamma=0.0, learning_rate=0.05, max_depth=3, min_child_weight=1, n_estimators=60 


...

[CV]  colsample_bytree=0.9, gamma=0.2, learning_rate=0.06, max_depth=3, min_child_weight=2, n_estimators=55, score=0.796, total=   0.0s
[CV] colsample_bytree=0.9, gamma=0.2, learning_rate=0.06, max_depth=3, min_child_weight=2, n_estimators=55 
[CV]  colsample_bytree=0.9, gamma=0.2, learning_rate=0.06, max_depth=3, min_child_weight=2, n_estimators=55, score=0.710, total=   0.0s
[CV] colsample_bytree=0.9, gamma=0.2, learning_rate=0.06, max_depth=3, min_child_weight=2, n_estimators=55 
[CV]  colsample_bytree=0.9, gamma=0.2, learning_rate=0.06, max_depth=3, min_child_weight=2, n_estimators=55, score=0.863, total=   0.1s
[CV] colsample_bytree=0.9, gamma=0.2, learning_rate=0.06, max_depth=3, min_child_weight=2, n_estimators=55 
[CV]  colsample_bytree=0.9, gamma=0.2, learning_rate=0.06, max_depth=3, min_child_weight=2, n_estimators=55, score=0.864, total=   0.0s
[CV] colsample_bytree=0.9, gamma=0.2, learning_rate=0.06, max_depth=3, min_child_weight=2, n_estimators=60 
[CV]  colsample_bytree=0

[CV]  colsample_bytree=0.9, gamma=0.2, learning_rate=0.06, max_depth=4, min_child_weight=1, n_estimators=50, score=0.769, total=   0.1s
[CV] colsample_bytree=0.9, gamma=0.2, learning_rate=0.06, max_depth=4, min_child_weight=1, n_estimators=50 
[CV]  colsample_bytree=0.9, gamma=0.2, learning_rate=0.06, max_depth=4, min_child_weight=1, n_estimators=50, score=0.709, total=   0.1s
[CV] colsample_bytree=0.9, gamma=0.2, learning_rate=0.06, max_depth=4, min_child_weight=1, n_estimators=50 
[CV]  colsample_bytree=0.9, gamma=0.2, learning_rate=0.06, max_depth=4, min_child_weight=1, n_estimators=50, score=0.870, total=   0.1s
[CV] colsample_bytree=0.9, gamma=0.2, learning_rate=0.06, max_depth=4, min_child_weight=1, n_estimators=50 
[CV]  colsample_bytree=0.9, gamma=0.2, learning_rate=0.06, max_depth=4, min_child_weight=1, n_estimators=50, score=0.880, total=   0.1s
[CV] colsample_bytree=0.9, gamma=0.2, learning_rate=0.06, max_depth=4, min_child_weight=1, n_estimators=55 
[CV]  colsample_bytree=0

[CV]  colsample_bytree=0.9, gamma=0.2, learning_rate=0.06, max_depth=4, min_child_weight=2, n_estimators=65, score=0.696, total=   0.1s
[CV] colsample_bytree=0.9, gamma=0.2, learning_rate=0.06, max_depth=4, min_child_weight=2, n_estimators=65 
[CV]  colsample_bytree=0.9, gamma=0.2, learning_rate=0.06, max_depth=4, min_child_weight=2, n_estimators=65, score=0.855, total=   0.1s
[CV] colsample_bytree=0.9, gamma=0.2, learning_rate=0.06, max_depth=4, min_child_weight=2, n_estimators=65 
[CV]  colsample_bytree=0.9, gamma=0.2, learning_rate=0.06, max_depth=4, min_child_weight=2, n_estimators=65, score=0.878, total=   0.1s
[CV] colsample_bytree=0.9, gamma=0.2, learning_rate=0.06, max_depth=4, min_child_weight=3, n_estimators=50 
[CV]  colsample_bytree=0.9, gamma=0.2, learning_rate=0.06, max_depth=4, min_child_weight=3, n_estimators=50, score=0.854, total=   0.1s
[CV] colsample_bytree=0.9, gamma=0.2, learning_rate=0.06, max_depth=4, min_child_weight=3, n_estimators=50 
[CV]  colsample_bytree=0

[Parallel(n_jobs=1)]: Done 2160 out of 2160 | elapsed:  2.3min finished


GridSearchCV(cv=5, error_score='raise-deprecating',
             estimator=XGBRegressor(base_score=None, booster=None,
                                    colsample_bylevel=None,
                                    colsample_bynode=None,
                                    colsample_bytree=None, gamma=None,
                                    gpu_id=None, importance_type='gain',
                                    interaction_constraints=None,
                                    learning_rate=None, max_delta_step=None,
                                    max_depth=None, min_child_weight=None,
                                    missing=nan, monotone_constrain...
                                    subsample=None, tree_method=None,
                                    validate_parameters=False, verbosity=None),
             iid='warn', n_jobs=None,
             param_grid={'colsample_bytree': [0.7, 0.8, 0.9],
                         'gamma': [0.0, 0.1, 0.2],
                         'le

In-sample Score

In [28]:
y_pred_train = grid_.best_estimator_.predict(X_train_sj)
score = mean_absolute_error(y_train_sj, y_pred_train)
print(score)

6.877645672741024


In [29]:
y_pred_train.shape

(651,)

Save feature importance as dataframe

In [32]:
feature_importance = pd.DataFrame(grid_.best_estimator_.feature_importances_, index = X_train_sj.columns ).rename(columns={0:'XGB'})

In [33]:
train_pred_sj = pd.DataFrame(y_pred_train, columns = ['xgb'])

In [34]:
grid_.best_estimator_.feature_importances_

array([0.01791015, 0.03618509, 0.01577843, 0.01258469, 0.01154439,
       0.01037049, 0.70904917, 0.18657768], dtype=float32)

Out-of-sample Score

In [35]:
def calculate_validation_score(model,X_val):
    '''
    -----
    Input:
        model (Regressor): trained model
        y (pd.DataFrame): y(label) dataset
    -----
    Output:
        y_pred_list (list): result of the prediction
        df_X_val (pd.DataFrame): validation set that was filled with predicted values as lag features 
  
    '''
    y_pred_list = []
    df_X_val = X_val.copy()
    for i in range(0,len(X_val)-2):
        predicts = []
        y_pred = model.predict(pd.DataFrame(df_X_val.drop(['year','weekofyear'], axis=1).iloc[i,:]).transpose())
        predicts.append(y_pred)
        for k in range(1,3):  
            df_X_val['total_cases_'+str(k)+'_week_ago'].iloc[i+k] = y_pred 
        y_pred_list.append(int(y_pred))
    y_pred = model.predict(pd.DataFrame(df_X_val.drop(['year','weekofyear'], axis=1).iloc[-2,:]).transpose())
    y_pred_list.append(int(y_pred))
    df_X_val['total_cases_1_week_ago'].iloc[-1] = y_pred
    y_pred = model.predict(pd.DataFrame(df_X_val.drop(['year','weekofyear'], axis=1).iloc[-1,:]).transpose())
    y_pred_list.append(int(y_pred))
    return y_pred_list, df_X_val

In [36]:
y_pred_val_sj = calculate_validation_score(grid_.best_estimator_ ,X_val_sj.drop('city', axis=1))
score = mean_absolute_error(y_val_sj, [x for x in y_pred_val_sj[0]])
print(score)

14.236559139784946


In [37]:
grid_.best_estimator_.feature_importances_

array([0.01791015, 0.03618509, 0.01577843, 0.01258469, 0.01154439,
       0.01037049, 0.70904917, 0.18657768], dtype=float32)

In [38]:
val_result_sj = y_pred_val_sj[1][['year','weekofyear']]

In [39]:
val_result_sj['XGB'] = y_pred_val_sj[0]

In [40]:
val_result_sj.head()

,year,weekofyear,XGB
0,2002,51,8
1,2002,52,10
2,2003,1,11
3,2003,2,13
4,2003,3,14


### IQ

In [41]:
X_train_iq.head()

,ndvi_sw,reanalysis_dew_point_temp_k,reanalysis_specific_humidity_g_per_kg,reanalysis_min_air_temp_k,station_min_temp_c,station_max_temp_c,total_cases_1_week_ago,total_cases_2_week_ago
651,0.200314,292.797143,14.431429,288.6,14.7,31.5,0,0
652,0.361043,293.957143,15.444286,291.5,19.1,33.3,0,0
653,0.255314,291.725714,13.421429,288.5,17.0,32.0,0,0
654,0.387271,293.821429,15.311429,292.1,19.9,34.0,0,0
655,0.382750,293.992857,15.465714,290.6,20.5,33.0,0,0


In [42]:
from sklearn.model_selection import GridSearchCV
from xgboost import XGBRegressor
xgb_regressor = XGBRegressor()
xgb_params = {'max_depth' : (3,4),
              'learning_rate': (0.05,0.06),
              'n_estimators' : (50,55,60,65),
             "min_child_weight" : [ 1, 2, 3],
             "gamma"            : [0.0,0.1, 0.2],
             "colsample_bytree" : [ 0.7,0.8,0.9] }

grid_  = GridSearchCV(xgb_regressor, xgb_params,cv=5, verbose=20, scoring= 'neg_mean_absolute_error') 
grid_.fit(X_train_iq,y_train_iq)

Fitting 5 folds for each of 432 candidates, totalling 2160 fits
[CV] colsample_bytree=0.7, gamma=0.0, learning_rate=0.05, max_depth=3, min_child_weight=1, n_estimators=50 
[CV]  colsample_bytree=0.7, gamma=0.0, learning_rate=0.05, max_depth=3, min_child_weight=1, n_estimators=50, score=-1.830, total=   0.0s
[CV] colsample_bytree=0.7, gamma=0.0, learning_rate=0.05, max_depth=3, min_child_weight=1, n_estimators=50 
[CV]  colsample_bytree=0.7, gamma=0.0, learning_rate=0.05, max_depth=3, min_child_weight=1, n_estimators=50, score=-5.505, total=   0.0s
[CV] colsample_bytree=0.7, gamma=0.0, learning_rate=0.05, max_depth=3, min_child_weight=1, n_estimators=50 
[CV]  colsample_bytree=0.7, gamma=0.0, learning_rate=0.05, max_depth=3, min_child_weight=1, n_estimators=50, score=-1.973, total=   0.0s
[CV] colsample_bytree=0.7, gamma=0.0, learning_rate=0.05, max_depth=3, min_child_weight=1, n_estimators=50 
[CV]  colsample_bytree=0.7, gamma=0.0, learning_rate=0.05, max_depth=3, min_child_weight=1, n

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   3 out of   3 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   4 out of   4 | elapsed:    0.1s remaining:    0.0s



[CV] colsample_bytree=0.7, gamma=0.0, learning_rate=0.05, max_depth=3, min_child_weight=1, n_estimators=50 
[CV]  colsample_bytree=0.7, gamma=0.0, learning_rate=0.05, max_depth=3, min_child_weight=1, n_estimators=50, score=-3.104, total=   0.0s
[CV] colsample_bytree=0.7, gamma=0.0, learning_rate=0.05, max_depth=3, min_child_weight=1, n_estimators=55 
[CV]  colsample_bytree=0.7, gamma=0.0, learning_rate=0.05, max_depth=3, min_child_weight=1, n_estimators=55, score=-1.772, total=   0.0s
[CV] colsample_bytree=0.7, gamma=0.0, learning_rate=0.05, max_depth=3, min_child_weight=1, n_estimators=55 
[CV]  colsample_bytree=0.7, gamma=0.0, learning_rate=0.05, max_depth=3, min_child_weight=1, n_estimators=55, score=-5.643, total=   0.0s
[CV] colsample_bytree=0.7, gamma=0.0, learning_rate=0.05, max_depth=3, min_child_weight=1, n_estimators=55 
[CV]  colsample_bytree=0.7, gamma=0.0, learning_rate=0.05, max_depth=3, min_child_weight=1, n_estimators=55, score=-1.994, total=   0.0s
[CV] colsample_bytr

[Parallel(n_jobs=1)]: Done   5 out of   5 | elapsed:    0.1s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   6 out of   6 | elapsed:    0.2s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   7 out of   7 | elapsed:    0.2s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   8 out of   8 | elapsed:    0.3s remaining:    0.0s


[CV]  colsample_bytree=0.7, gamma=0.0, learning_rate=0.05, max_depth=3, min_child_weight=1, n_estimators=55, score=-6.848, total=   0.0s
[CV] colsample_bytree=0.7, gamma=0.0, learning_rate=0.05, max_depth=3, min_child_weight=1, n_estimators=55 
[CV]  colsample_bytree=0.7, gamma=0.0, learning_rate=0.05, max_depth=3, min_child_weight=1, n_estimators=55, score=-3.140, total=   0.0s
[CV] colsample_bytree=0.7, gamma=0.0, learning_rate=0.05, max_depth=3, min_child_weight=1, n_estimators=60 
[CV]  colsample_bytree=0.7, gamma=0.0, learning_rate=0.05, max_depth=3, min_child_weight=1, n_estimators=60, score=-1.803, total=   0.0s
[CV] colsample_bytree=0.7, gamma=0.0, learning_rate=0.05, max_depth=3, min_child_weight=1, n_estimators=60 
[CV]  colsample_bytree=0.7, gamma=0.0, learning_rate=0.05, max_depth=3, min_child_weight=1, n_estimators=60, score=-5.770, total=   0.1s
[CV] colsample_bytree=0.7, gamma=0.0, learning_rate=0.05, max_depth=3, min_child_weight=1, n_estimators=60 
[CV]  colsample_bytr

...

[CV]  colsample_bytree=0.9, gamma=0.2, learning_rate=0.05, max_depth=4, min_child_weight=3, n_estimators=60, score=-2.025, total=   0.1s
[CV] colsample_bytree=0.9, gamma=0.2, learning_rate=0.05, max_depth=4, min_child_weight=3, n_estimators=60 
[CV]  colsample_bytree=0.9, gamma=0.2, learning_rate=0.05, max_depth=4, min_child_weight=3, n_estimators=60, score=-6.772, total=   0.1s
[CV] colsample_bytree=0.9, gamma=0.2, learning_rate=0.05, max_depth=4, min_child_weight=3, n_estimators=60 
[CV]  colsample_bytree=0.9, gamma=0.2, learning_rate=0.05, max_depth=4, min_child_weight=3, n_estimators=60, score=-3.662, total=   0.1s
[CV] colsample_bytree=0.9, gamma=0.2, learning_rate=0.05, max_depth=4, min_child_weight=3, n_estimators=65 
[CV]  colsample_bytree=0.9, gamma=0.2, learning_rate=0.05, max_depth=4, min_child_weight=3, n_estimators=65, score=-1.502, total=   0.1s
[CV] colsample_bytree=0.9, gamma=0.2, learning_rate=0.05, max_depth=4, min_child_weight=3, n_estimators=65 
[CV]  colsample_bytr

[CV]  colsample_bytree=0.9, gamma=0.2, learning_rate=0.06, max_depth=3, min_child_weight=2, n_estimators=55, score=-3.088, total=   0.0s
[CV] colsample_bytree=0.9, gamma=0.2, learning_rate=0.06, max_depth=3, min_child_weight=2, n_estimators=60 
[CV]  colsample_bytree=0.9, gamma=0.2, learning_rate=0.06, max_depth=3, min_child_weight=2, n_estimators=60, score=-2.375, total=   0.0s
[CV] colsample_bytree=0.9, gamma=0.2, learning_rate=0.06, max_depth=3, min_child_weight=2, n_estimators=60 
[CV]  colsample_bytree=0.9, gamma=0.2, learning_rate=0.06, max_depth=3, min_child_weight=2, n_estimators=60, score=-5.321, total=   0.0s
[CV] colsample_bytree=0.9, gamma=0.2, learning_rate=0.06, max_depth=3, min_child_weight=2, n_estimators=60 
[CV]  colsample_bytree=0.9, gamma=0.2, learning_rate=0.06, max_depth=3, min_child_weight=2, n_estimators=60, score=-1.970, total=   0.0s
[CV] colsample_bytree=0.9, gamma=0.2, learning_rate=0.06, max_depth=3, min_child_weight=2, n_estimators=60 
[CV]  colsample_bytr

[CV]  colsample_bytree=0.9, gamma=0.2, learning_rate=0.06, max_depth=4, min_child_weight=1, n_estimators=55, score=-6.000, total=   0.0s
[CV] colsample_bytree=0.9, gamma=0.2, learning_rate=0.06, max_depth=4, min_child_weight=1, n_estimators=55 
[CV]  colsample_bytree=0.9, gamma=0.2, learning_rate=0.06, max_depth=4, min_child_weight=1, n_estimators=55, score=-2.000, total=   0.0s
[CV] colsample_bytree=0.9, gamma=0.2, learning_rate=0.06, max_depth=4, min_child_weight=1, n_estimators=55 
[CV]  colsample_bytree=0.9, gamma=0.2, learning_rate=0.06, max_depth=4, min_child_weight=1, n_estimators=55, score=-6.927, total=   0.1s
[CV] colsample_bytree=0.9, gamma=0.2, learning_rate=0.06, max_depth=4, min_child_weight=1, n_estimators=55 
[CV]  colsample_bytree=0.9, gamma=0.2, learning_rate=0.06, max_depth=4, min_child_weight=1, n_estimators=55, score=-3.478, total=   0.0s
[CV] colsample_bytree=0.9, gamma=0.2, learning_rate=0.06, max_depth=4, min_child_weight=1, n_estimators=60 
[CV]  colsample_bytr

[Parallel(n_jobs=1)]: Done 2160 out of 2160 | elapsed:  2.0min finished


GridSearchCV(cv=5, error_score='raise-deprecating',
             estimator=XGBRegressor(base_score=None, booster=None,
                                    colsample_bylevel=None,
                                    colsample_bynode=None,
                                    colsample_bytree=None, gamma=None,
                                    gpu_id=None, importance_type='gain',
                                    interaction_constraints=None,
                                    learning_rate=None, max_delta_step=None,
                                    max_depth=None, min_child_weight=None,
                                    missing=nan, monotone_constrain...
                                    subsample=None, tree_method=None,
                                    validate_parameters=False, verbosity=None),
             iid='warn', n_jobs=None,
             param_grid={'colsample_bytree': [0.7, 0.8, 0.9],
                         'gamma': [0.0, 0.1, 0.2],
                         'le

In-sample score

In [43]:
y_pred_train = grid_.best_estimator_.predict(X_train_iq)
score = mean_absolute_error(y_train_iq, y_pred_train)
print(score)

2.879276339722211


In [44]:
train_pred_iq = pd.DataFrame(y_pred_train, columns = ['xgb'])

Out-of-sample score

In [45]:
y_pred_val_iq = calculate_validation_score(grid_.best_estimator_ ,X_val_iq.drop('city', axis=1))
score = mean_absolute_error(y_val_iq, [x for x in y_pred_val_iq[0]])
print(score)

7.587096774193548


In [46]:
val_result_iq = y_pred_val_iq[1][['year','weekofyear']]
val_result_iq['XGB'] = y_pred_val_iq[0]

## SVM

### SJ

In [47]:
from sklearn.svm import SVR
param_grid ={'kernel':["rbf"],
            'C':[np.exp(i) for i in np.linspace(-5,5,100)],
            "gamma": np.logspace(-2, 2, 5)}
svr=SVR()
grid_=GridSearchCV(svr, cv=5, param_grid=param_grid, verbose=3, scoring= 'neg_mean_absolute_error')
grid_.fit(X_train_sj,y_train_sj)

Fitting 5 folds for each of 500 candidates, totalling 2500 fits
[CV] C=0.006737946999085467, gamma=0.01, kernel=rbf ..................
[CV]  C=0.006737946999085467, gamma=0.01, kernel=rbf, score=-27.473, total=   0.1s
[CV] C=0.006737946999085467, gamma=0.01, kernel=rbf ..................
[CV]  C=0.006737946999085467, gamma=0.01, kernel=rbf, score=-49.309, total=   0.0s
[CV] C=0.006737946999085467, gamma=0.01, kernel=rbf ..................
[CV]  C=0.006737946999085467, gamma=0.01, kernel=rbf, score=-12.571, total=   0.0s
[CV] C=0.006737946999085467, gamma=0.01, kernel=rbf ..................
[CV]  C=0.006737946999085467, gamma=0.01, kernel=rbf, score=-36.309, total=   0.0s
[CV] C=0.006737946999085467, gamma=0.01, kernel=rbf ..................
[CV]  C=0.006737946999085467, gamma=0.01, kernel=rbf, score=-17.976, total=   0.0s
[CV] C=0.006737946999085467, gamma=0.1, kernel=rbf ...................


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    0.0s remaining:    0.0s


[CV]  C=0.006737946999085467, gamma=0.1, kernel=rbf, score=-27.774, total=   0.0s
[CV] C=0.006737946999085467, gamma=0.1, kernel=rbf ...................
[CV]  C=0.006737946999085467, gamma=0.1, kernel=rbf, score=-49.487, total=   0.0s
[CV] C=0.006737946999085467, gamma=0.1, kernel=rbf ...................
[CV]  C=0.006737946999085467, gamma=0.1, kernel=rbf, score=-12.745, total=   0.0s
[CV] C=0.006737946999085467, gamma=0.1, kernel=rbf ...................
[CV]  C=0.006737946999085467, gamma=0.1, kernel=rbf, score=-36.511, total=   0.0s
[CV] C=0.006737946999085467, gamma=0.1, kernel=rbf ...................
[CV]  C=0.006737946999085467, gamma=0.1, kernel=rbf, score=-18.141, total=   0.0s
[CV] C=0.006737946999085467, gamma=1.0, kernel=rbf ...................
[CV]  C=0.006737946999085467, gamma=1.0, kernel=rbf, score=-27.788, total=   0.1s
[CV] C=0.006737946999085467, gamma=1.0, kernel=rbf ...................
[CV]  C=0.006737946999085467, gamma=1.0, kernel=rbf, score=-49.503, total=   0.1s


[CV]  C=0.008246389605637307, gamma=0.1, kernel=rbf, score=-18.136, total=   0.1s
[CV] C=0.008246389605637307, gamma=1.0, kernel=rbf ...................
[CV]  C=0.008246389605637307, gamma=1.0, kernel=rbf, score=-27.788, total=   0.1s
[CV] C=0.008246389605637307, gamma=1.0, kernel=rbf ...................
[CV]  C=0.008246389605637307, gamma=1.0, kernel=rbf, score=-49.503, total=   0.1s
[CV] C=0.008246389605637307, gamma=1.0, kernel=rbf ...................
[CV]  C=0.008246389605637307, gamma=1.0, kernel=rbf, score=-12.763, total=   0.1s
[CV] C=0.008246389605637307, gamma=1.0, kernel=rbf ...................
[CV]  C=0.008246389605637307, gamma=1.0, kernel=rbf, score=-36.515, total=   0.1s
[CV] C=0.008246389605637307, gamma=1.0, kernel=rbf ...................
[CV]  C=0.008246389605637307, gamma=1.0, kernel=rbf, score=-18.164, total=   0.1s
[CV] C=0.008246389605637307, gamma=10.0, kernel=rbf ..................
[CV]  C=0.008246389605637307, gamma=10.0, kernel=rbf, score=-27.788, total=   0.1s

[CV]  C=0.010092531380432784, gamma=10.0, kernel=rbf, score=-27.788, total=   0.1s
[CV] C=0.010092531380432784, gamma=10.0, kernel=rbf ..................
[CV]  C=0.010092531380432784, gamma=10.0, kernel=rbf, score=-49.504, total=   0.1s
[CV] C=0.010092531380432784, gamma=10.0, kernel=rbf ..................
[CV]  C=0.010092531380432784, gamma=10.0, kernel=rbf, score=-12.764, total=   0.1s
[CV] C=0.010092531380432784, gamma=10.0, kernel=rbf ..................
[CV]  C=0.010092531380432784, gamma=10.0, kernel=rbf, score=-36.515, total=   0.1s
[CV] C=0.010092531380432784, gamma=10.0, kernel=rbf ..................
[CV]  C=0.010092531380432784, gamma=10.0, kernel=rbf, score=-18.165, total=   0.1s
[CV] C=0.010092531380432784, gamma=100.0, kernel=rbf .................
[CV]  C=0.010092531380432784, gamma=100.0, kernel=rbf, score=-27.788, total=   0.1s
[CV] C=0.010092531380432784, gamma=100.0, kernel=rbf .................
[CV]  C=0.010092531380432784, gamma=100.0, kernel=rbf, score=-49.504, total

...

[CV]  C=80.95871990300357, gamma=1.0, kernel=rbf, score=-18.389, total=   0.1s
[CV] C=80.95871990300357, gamma=1.0, kernel=rbf ......................
[CV]  C=80.95871990300357, gamma=1.0, kernel=rbf, score=-34.015, total=   0.1s
[CV] C=80.95871990300357, gamma=1.0, kernel=rbf ......................
[CV]  C=80.95871990300357, gamma=1.0, kernel=rbf, score=-24.216, total=   0.1s
[CV] C=80.95871990300357, gamma=10.0, kernel=rbf .....................
[CV]  C=80.95871990300357, gamma=10.0, kernel=rbf, score=-25.448, total=   0.1s
[CV] C=80.95871990300357, gamma=10.0, kernel=rbf .....................
[CV]  C=80.95871990300357, gamma=10.0, kernel=rbf, score=-49.653, total=   0.1s
[CV] C=80.95871990300357, gamma=10.0, kernel=rbf .....................
[CV]  C=80.95871990300357, gamma=10.0, kernel=rbf, score=-18.668, total=   0.1s
[CV] C=80.95871990300357, gamma=10.0, kernel=rbf .....................
[CV]  C=80.95871990300357, gamma=10.0, kernel=rbf, score=-34.492, total=   0.1s
[CV] C=80.9587199

[CV]  C=99.08316975251431, gamma=10.0, kernel=rbf, score=-34.350, total=   0.1s
[CV] C=99.08316975251431, gamma=10.0, kernel=rbf .....................
[CV]  C=99.08316975251431, gamma=10.0, kernel=rbf, score=-26.045, total=   0.1s
[CV] C=99.08316975251431, gamma=100.0, kernel=rbf ....................
[CV]  C=99.08316975251431, gamma=100.0, kernel=rbf, score=-25.414, total=   0.1s
[CV] C=99.08316975251431, gamma=100.0, kernel=rbf ....................
[CV]  C=99.08316975251431, gamma=100.0, kernel=rbf, score=-49.814, total=   0.1s
[CV] C=99.08316975251431, gamma=100.0, kernel=rbf ....................
[CV]  C=99.08316975251431, gamma=100.0, kernel=rbf, score=-19.669, total=   0.1s
[CV] C=99.08316975251431, gamma=100.0, kernel=rbf ....................
[CV]  C=99.08316975251431, gamma=100.0, kernel=rbf, score=-34.350, total=   0.1s
[CV] C=99.08316975251431, gamma=100.0, kernel=rbf ....................
[CV]  C=99.08316975251431, gamma=100.0, kernel=rbf, score=-26.044, total=   0.1s
[CV] C=10

[CV]  C=134.15420192797174, gamma=0.01, kernel=rbf, score=-12.931, total=   0.1s
[CV] C=134.15420192797174, gamma=0.01, kernel=rbf ....................
[CV]  C=134.15420192797174, gamma=0.01, kernel=rbf, score=-35.052, total=   0.1s
[CV] C=134.15420192797174, gamma=0.01, kernel=rbf ....................
[CV]  C=134.15420192797174, gamma=0.01, kernel=rbf, score=-7.275, total=   0.1s
[CV] C=134.15420192797174, gamma=0.01, kernel=rbf ....................
[CV]  C=134.15420192797174, gamma=0.01, kernel=rbf, score=-17.069, total=   0.1s
[CV] C=134.15420192797174, gamma=0.01, kernel=rbf ....................
[CV]  C=134.15420192797174, gamma=0.01, kernel=rbf, score=-5.995, total=   0.1s
[CV] C=134.15420192797174, gamma=0.1, kernel=rbf .....................
[CV]  C=134.15420192797174, gamma=0.1, kernel=rbf, score=-16.503, total=   0.1s
[CV] C=134.15420192797174, gamma=0.1, kernel=rbf .....................
[CV]  C=134.15420192797174, gamma=0.1, kernel=rbf, score=-43.417, total=   0.1s
[CV] C=134.

[Parallel(n_jobs=1)]: Done 2500 out of 2500 | elapsed:  3.1min finished


GridSearchCV(cv=5, error_score='raise-deprecating',
             estimator=SVR(C=1.0, cache_size=200, coef0=0.0, degree=3,
                           epsilon=0.1, gamma='auto_deprecated', kernel='rbf',
                           max_iter=-1, shrinking=True, tol=0.001,
                           verbose=False),
             iid='warn', n_jobs=None,
             param_grid={'C': [0.006737946999085467, 0.0074541086721749295,
                               0.008246389605637307, 0.009122880349438393,
                               0.010092531380432784...
                               0.050803096080009554, 0.05620284622522774,
                               0.06217652402211632, 0.06878513098750348,
                               0.07609615235623413, 0.08418424622139925,
                               0.09313200592177617, 0.10303080346176416,
                               0.11398172257658408, 0.1260965909709399, ...],
                         'gamma': array([1.e-02, 1.e-01, 1.e+00, 1.e+01, 

In-sample score

In [48]:
y_pred_train = grid_.best_estimator_.predict(X_train_sj)
score = mean_absolute_error(y_train_sj, y_pred_train)
print(score)

7.026212003433041


In [49]:
train_pred_sj['SVM'] = y_pred_train

Out-of-sample

In [50]:
y_pred_val_sj = calculate_validation_score(grid_.best_estimator_ ,X_val_sj.drop('city', axis=1))
score = mean_absolute_error(y_val_sj, [x for x in y_pred_val_sj[0]])
print(score)

51.913978494623656


In [51]:
val_result_sj['SVM'] = y_pred_val_sj[0]

### IQ

In [52]:
from sklearn.svm import SVR
param_grid ={'kernel':["rbf"],
            'C':[np.exp(i) for i in np.linspace(-5,5,100)],
            "gamma": np.logspace(-2, 2, 5)}
svr=SVR()
grid_=GridSearchCV(svr, cv=5, param_grid=param_grid, verbose=3)
grid_.fit(X_train_iq,y_train_iq)

Fitting 5 folds for each of 500 candidates, totalling 2500 fits
[CV] C=0.006737946999085467, gamma=0.01, kernel=rbf ..................
[CV]  C=0.006737946999085467, gamma=0.01, kernel=rbf, score=-77.282, total=   0.0s
[CV] C=0.006737946999085467, gamma=0.01, kernel=rbf ..................
[CV]  C=0.006737946999085467, gamma=0.01, kernel=rbf, score=-0.673, total=   0.0s
[CV] C=0.006737946999085467, gamma=0.01, kernel=rbf ..................
[CV]  C=0.006737946999085467, gamma=0.01, kernel=rbf, score=0.061, total=   0.0s
[CV] C=0.006737946999085467, gamma=0.01, kernel=rbf ..................
[CV]  C=0.006737946999085467, gamma=0.01, kernel=rbf, score=-0.265, total=   0.0s
[CV] C=0.006737946999085467, gamma=0.01, kernel=rbf ..................
[CV]  C=0.006737946999085467, gamma=0.01, kernel=rbf, score=-0.107, total=   0.0s
[CV] C=0.006737946999085467, gamma=0.1, kernel=rbf ...................
[CV]  C=0.006737946999085467, gamma=0.1, kernel=rbf, score=-81.165, total=   0.0s
[CV] C=0.006737946

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    0.0s remaining:    0.0s



[CV]  C=0.006737946999085467, gamma=0.1, kernel=rbf, score=-0.153, total=   0.0s
[CV] C=0.006737946999085467, gamma=1.0, kernel=rbf ...................
[CV]  C=0.006737946999085467, gamma=1.0, kernel=rbf, score=-83.417, total=   0.0s
[CV] C=0.006737946999085467, gamma=1.0, kernel=rbf ...................
[CV]  C=0.006737946999085467, gamma=1.0, kernel=rbf, score=-0.722, total=   0.0s
[CV] C=0.006737946999085467, gamma=1.0, kernel=rbf ...................
[CV]  C=0.006737946999085467, gamma=1.0, kernel=rbf, score=-0.038, total=   0.0s
[CV] C=0.006737946999085467, gamma=1.0, kernel=rbf ...................
[CV]  C=0.006737946999085467, gamma=1.0, kernel=rbf, score=-0.280, total=   0.0s
[CV] C=0.006737946999085467, gamma=1.0, kernel=rbf ...................
[CV]  C=0.006737946999085467, gamma=1.0, kernel=rbf, score=-0.165, total=   0.0s
[CV] C=0.006737946999085467, gamma=10.0, kernel=rbf ..................
[CV]  C=0.006737946999085467, gamma=10.0, kernel=rbf, score=-83.454, total=   0.0s
[CV

[CV]  C=0.008246389605637307, gamma=1.0, kernel=rbf, score=-0.280, total=   0.0s
[CV] C=0.008246389605637307, gamma=1.0, kernel=rbf ...................
[CV]  C=0.008246389605637307, gamma=1.0, kernel=rbf, score=-0.165, total=   0.0s
[CV] C=0.008246389605637307, gamma=10.0, kernel=rbf ..................
[CV]  C=0.008246389605637307, gamma=10.0, kernel=rbf, score=-83.427, total=   0.0s
[CV] C=0.008246389605637307, gamma=10.0, kernel=rbf ..................
[CV]  C=0.008246389605637307, gamma=10.0, kernel=rbf, score=-0.722, total=   0.0s
[CV] C=0.008246389605637307, gamma=10.0, kernel=rbf ..................
[CV]  C=0.008246389605637307, gamma=10.0, kernel=rbf, score=-0.038, total=   0.0s
[CV] C=0.008246389605637307, gamma=10.0, kernel=rbf ..................
[CV]  C=0.008246389605637307, gamma=10.0, kernel=rbf, score=-0.280, total=   0.0s
[CV] C=0.008246389605637307, gamma=10.0, kernel=rbf ..................
[CV]  C=0.008246389605637307, gamma=10.0, kernel=rbf, score=-0.165, total=   0.0s
[

[CV]  C=0.010092531380432784, gamma=10.0, kernel=rbf, score=-0.038, total=   0.0s
[CV] C=0.010092531380432784, gamma=10.0, kernel=rbf ..................
[CV]  C=0.010092531380432784, gamma=10.0, kernel=rbf, score=-0.280, total=   0.0s
[CV] C=0.010092531380432784, gamma=10.0, kernel=rbf ..................
[CV]  C=0.010092531380432784, gamma=10.0, kernel=rbf, score=-0.165, total=   0.0s
[CV] C=0.010092531380432784, gamma=100.0, kernel=rbf .................
[CV]  C=0.010092531380432784, gamma=100.0, kernel=rbf, score=-83.394, total=   0.0s
[CV] C=0.010092531380432784, gamma=100.0, kernel=rbf .................
[CV]  C=0.010092531380432784, gamma=100.0, kernel=rbf, score=-0.722, total=   0.0s
[CV] C=0.010092531380432784, gamma=100.0, kernel=rbf .................
[CV]  C=0.010092531380432784, gamma=100.0, kernel=rbf, score=-0.038, total=   0.0s
[CV] C=0.010092531380432784, gamma=100.0, kernel=rbf .................
[CV]  C=0.010092531380432784, gamma=100.0, kernel=rbf, score=-0.280, total=   

...

[CV]  C=99.08316975251431, gamma=0.01, kernel=rbf, score=0.156, total=   0.0s
[CV] C=99.08316975251431, gamma=0.01, kernel=rbf .....................
[CV]  C=99.08316975251431, gamma=0.01, kernel=rbf, score=0.222, total=   0.0s
[CV] C=99.08316975251431, gamma=0.01, kernel=rbf .....................
[CV]  C=99.08316975251431, gamma=0.01, kernel=rbf, score=0.079, total=   0.0s
[CV] C=99.08316975251431, gamma=0.01, kernel=rbf .....................
[CV]  C=99.08316975251431, gamma=0.01, kernel=rbf, score=-0.049, total=   0.1s
[CV] C=99.08316975251431, gamma=0.1, kernel=rbf ......................
[CV]  C=99.08316975251431, gamma=0.1, kernel=rbf, score=-26.171, total=   0.1s
[CV] C=99.08316975251431, gamma=0.1, kernel=rbf ......................
[CV]  C=99.08316975251431, gamma=0.1, kernel=rbf, score=0.254, total=   0.1s
[CV] C=99.08316975251431, gamma=0.1, kernel=rbf ......................
[CV]  C=99.08316975251431, gamma=0.1, kernel=rbf, score=-0.066, total=   0.1s
[CV] C=99.08316975251431, g

[CV]  C=121.2651897160414, gamma=0.1, kernel=rbf, score=-0.097, total=   0.1s
[CV] C=121.2651897160414, gamma=0.1, kernel=rbf ......................
[CV]  C=121.2651897160414, gamma=0.1, kernel=rbf, score=0.005, total=   0.1s
[CV] C=121.2651897160414, gamma=0.1, kernel=rbf ......................
[CV]  C=121.2651897160414, gamma=0.1, kernel=rbf, score=-0.222, total=   0.1s
[CV] C=121.2651897160414, gamma=1.0, kernel=rbf ......................
[CV]  C=121.2651897160414, gamma=1.0, kernel=rbf, score=-144.895, total=   0.0s
[CV] C=121.2651897160414, gamma=1.0, kernel=rbf ......................
[CV]  C=121.2651897160414, gamma=1.0, kernel=rbf, score=-0.186, total=   0.0s
[CV] C=121.2651897160414, gamma=1.0, kernel=rbf ......................
[CV]  C=121.2651897160414, gamma=1.0, kernel=rbf, score=-0.988, total=   0.0s
[CV] C=121.2651897160414, gamma=1.0, kernel=rbf ......................
[CV]  C=121.2651897160414, gamma=1.0, kernel=rbf, score=-0.128, total=   0.0s
[CV] C=121.2651897160414, g

[CV]  C=148.4131591025766, gamma=10.0, kernel=rbf, score=-169.144, total=   0.0s
[CV] C=148.4131591025766, gamma=10.0, kernel=rbf .....................
[CV]  C=148.4131591025766, gamma=10.0, kernel=rbf, score=-0.300, total=   0.0s
[CV] C=148.4131591025766, gamma=10.0, kernel=rbf .....................
[CV]  C=148.4131591025766, gamma=10.0, kernel=rbf, score=-0.773, total=   0.0s
[CV] C=148.4131591025766, gamma=10.0, kernel=rbf .....................
[CV]  C=148.4131591025766, gamma=10.0, kernel=rbf, score=-0.163, total=   0.0s
[CV] C=148.4131591025766, gamma=10.0, kernel=rbf .....................
[CV]  C=148.4131591025766, gamma=10.0, kernel=rbf, score=-0.029, total=   0.0s
[CV] C=148.4131591025766, gamma=100.0, kernel=rbf ....................
[CV]  C=148.4131591025766, gamma=100.0, kernel=rbf, score=-169.195, total=   0.0s
[CV] C=148.4131591025766, gamma=100.0, kernel=rbf ....................
[CV]  C=148.4131591025766, gamma=100.0, kernel=rbf, score=-0.302, total=   0.0s
[CV] C=148.4131

[Parallel(n_jobs=1)]: Done 2500 out of 2500 | elapsed:  1.3min finished


GridSearchCV(cv=5, error_score='raise-deprecating',
             estimator=SVR(C=1.0, cache_size=200, coef0=0.0, degree=3,
                           epsilon=0.1, gamma='auto_deprecated', kernel='rbf',
                           max_iter=-1, shrinking=True, tol=0.001,
                           verbose=False),
             iid='warn', n_jobs=None,
             param_grid={'C': [0.006737946999085467, 0.0074541086721749295,
                               0.008246389605637307, 0.009122880349438393,
                               0.010092531380432784...
                               0.050803096080009554, 0.05620284622522774,
                               0.06217652402211632, 0.06878513098750348,
                               0.07609615235623413, 0.08418424622139925,
                               0.09313200592177617, 0.10303080346176416,
                               0.11398172257658408, 0.1260965909709399, ...],
                         'gamma': array([1.e-02, 1.e-01, 1.e+00, 1.e+01, 

In-sample score

In [53]:
y_pred_train = grid_.best_estimator_.predict(X_train_iq)
score = mean_absolute_error(y_train_iq, y_pred_train)
print(score)

2.6472563119866437


In [54]:
train_pred_iq['SVM'] = y_pred_train

Out-of-sample score

In [55]:
y_pred_val_iq = calculate_validation_score(grid_.best_estimator_ ,X_val_iq.drop('city', axis=1))
score = mean_absolute_error(y_val_iq, [x for x in y_pred_val_iq[0]])
print(score)

7.967741935483871


In [56]:
val_result_iq['SVM'] = y_pred_val_iq[0]

## Random Forest

## SJ

In [57]:
from sklearn.ensemble import RandomForestRegressor

rand_ = RandomForestRegressor(bootstrap=True)
rand_params = {    
    'max_depth': (7,8,9,10),
    'n_estimators':(70,80,90,100,200,400,600),
    'min_samples_leaf':(3,4,5)
}

grid_=GridSearchCV(rand_, cv=5, param_grid=rand_params, verbose=3)
grid_.fit(X_train_sj,y_train_sj)

Fitting 5 folds for each of 84 candidates, totalling 420 fits
[CV] max_depth=7, min_samples_leaf=3, n_estimators=70 ................


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


[CV]  max_depth=7, min_samples_leaf=3, n_estimators=70, score=0.858, total=   0.4s
[CV] max_depth=7, min_samples_leaf=3, n_estimators=70 ................


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.3s remaining:    0.0s


[CV]  max_depth=7, min_samples_leaf=3, n_estimators=70, score=0.822, total=   0.4s
[CV] max_depth=7, min_samples_leaf=3, n_estimators=70 ................


...

[CV]  max_depth=9, min_samples_leaf=4, n_estimators=400, score=0.860, total=   2.2s
[CV] max_depth=9, min_samples_leaf=4, n_estimators=400 ...............
[CV]  max_depth=9, min_samples_leaf=4, n_estimators=400, score=0.768, total=   2.0s
[CV] max_depth=9, min_samples_leaf=4, n_estimators=400 ...............
[CV]  max_depth=9, min_samples_leaf=4, n_estimators=400, score=0.692, total=   2.1s
[CV] max_depth=9, min_samples_leaf=4, n_estimators=400 ...............
[CV]  max_depth=9, min_samples_leaf=4, n_estimators=400, score=0.851, total=   2.2s
[CV] max_depth=9, min_samples_leaf=4, n_estimators=400 ...............
[CV]  max_depth=9, min_samples_leaf=4, n_estimators=400, score=0.868, total=   2.0s
[CV] max_depth=9, min_samples_leaf=4, n_estimators=600 ...............
[CV]  max_depth=9, min_samples_leaf=4, n_estimators=600, score=0.857, total=   3.1s
[CV] max_depth=9, min_samples_leaf=4, n_estimators=600 ...............
[CV]  max_depth=9, min_samples_leaf=4, n_estimators=600, score=0.774, 

[CV]  max_depth=10, min_samples_leaf=3, n_estimators=80, score=0.849, total=   0.5s
[CV] max_depth=10, min_samples_leaf=3, n_estimators=80 ...............
[CV]  max_depth=10, min_samples_leaf=3, n_estimators=80, score=0.869, total=   0.5s
[CV] max_depth=10, min_samples_leaf=3, n_estimators=90 ...............
[CV]  max_depth=10, min_samples_leaf=3, n_estimators=90, score=0.847, total=   0.5s
[CV] max_depth=10, min_samples_leaf=3, n_estimators=90 ...............
[CV]  max_depth=10, min_samples_leaf=3, n_estimators=90, score=0.812, total=   0.5s
[CV] max_depth=10, min_samples_leaf=3, n_estimators=90 ...............
[CV]  max_depth=10, min_samples_leaf=3, n_estimators=90, score=0.689, total=   0.5s
[CV] max_depth=10, min_samples_leaf=3, n_estimators=90 ...............
[CV]  max_depth=10, min_samples_leaf=3, n_estimators=90, score=0.846, total=   0.5s
[CV] max_depth=10, min_samples_leaf=3, n_estimators=90 ...............
[CV]  max_depth=10, min_samples_leaf=3, n_estimators=90, score=0.865, 

[CV]  max_depth=10, min_samples_leaf=4, n_estimators=400, score=0.775, total=   2.0s
[CV] max_depth=10, min_samples_leaf=4, n_estimators=400 ..............
[CV]  max_depth=10, min_samples_leaf=4, n_estimators=400, score=0.703, total=   2.1s
[CV] max_depth=10, min_samples_leaf=4, n_estimators=400 ..............
[CV]  max_depth=10, min_samples_leaf=4, n_estimators=400, score=0.853, total=   2.0s
[CV] max_depth=10, min_samples_leaf=4, n_estimators=400 ..............
[CV]  max_depth=10, min_samples_leaf=4, n_estimators=400, score=0.867, total=   2.1s
[CV] max_depth=10, min_samples_leaf=4, n_estimators=600 ..............
[CV]  max_depth=10, min_samples_leaf=4, n_estimators=600, score=0.856, total=   3.2s
[CV] max_depth=10, min_samples_leaf=4, n_estimators=600 ..............
[CV]  max_depth=10, min_samples_leaf=4, n_estimators=600, score=0.776, total=   3.2s
[CV] max_depth=10, min_samples_leaf=4, n_estimators=600 ..............
[CV]  max_depth=10, min_samples_leaf=4, n_estimators=600, score=

[Parallel(n_jobs=1)]: Done 420 out of 420 | elapsed:  8.3min finished


GridSearchCV(cv=5, error_score='raise-deprecating',
             estimator=RandomForestRegressor(bootstrap=True, criterion='mse',
                                             max_depth=None,
                                             max_features='auto',
                                             max_leaf_nodes=None,
                                             min_impurity_decrease=0.0,
                                             min_impurity_split=None,
                                             min_samples_leaf=1,
                                             min_samples_split=2,
                                             min_weight_fraction_leaf=0.0,
                                             n_estimators='warn', n_jobs=None,
                                             oob_score=False, random_state=None,
                                             verbose=0, warm_start=False),
             iid='warn', n_jobs=None,
             param_grid={'max_depth': (7, 8, 9, 10),
   

In-sample score

In [58]:
y_pred_train = grid_.best_estimator_.predict(X_train_sj)
score = mean_absolute_error(y_train_sj, y_pred_train)
print(score)

5.269320226300574


In [59]:
train_pred_sj['RF'] = y_pred_train

Out-of-sample score

In [60]:
y_pred_val_sj = calculate_validation_score(grid_.best_estimator_ ,X_val_sj.drop('city', axis=1))
score = mean_absolute_error(y_val_sj, [x for x in y_pred_val_sj[0]])
print(score)

19.74551971326165


In [61]:
val_result_sj['RF'] = y_pred_val_sj[0]

In [62]:
feature_importance['RF'] = grid_.best_estimator_.feature_importances_
feature_importance.to_csv('fi_approach2.csv')

## IQ

In [55]:
from sklearn.svm import SVR
rand_ = RandomForestRegressor(bootstrap=True)
rand_params = {    
    'max_depth': (7,8,9,10),
    'n_estimators':(70,80,90,100,200,400,600),
    'min_samples_leaf':(3,4,5)
}

grid_=GridSearchCV(rand_, cv=5, param_grid=rand_params, verbose=3)
grid_.fit(X_train_iq,y_train_iq)

Fitting 5 folds for each of 84 candidates, totalling 420 fits
[CV] max_depth=7, min_samples_leaf=3, n_estimators=70 ................
[CV]  max_depth=7, min_samples_leaf=3, n_estimators=70, score=-3.876, total=   0.1s
[CV] max_depth=7, min_samples_leaf=3, n_estimators=70 ................


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s


[CV]  max_depth=7, min_samples_leaf=3, n_estimators=70, score=0.229, total=   0.1s
[CV] max_depth=7, min_samples_leaf=3, n_estimators=70 ................
[CV]  max_depth=7, min_samples_leaf=3, n_estimators=70, score=0.366, total=   0.1s
[CV] max_depth=7, min_samples_leaf=3, n_estimators=70 ................


[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    0.2s remaining:    0.0s


[CV]  max_depth=7, min_samples_leaf=3, n_estimators=70, score=0.226, total=   0.1s
[CV] max_depth=7, min_samples_leaf=3, n_estimators=70 ................
[CV]  max_depth=7, min_samples_leaf=3, n_estimators=70, score=0.203, total=   0.2s
[CV] max_depth=7, min_samples_leaf=3, n_estimators=80 ................
[CV]  max_depth=7, min_samples_leaf=3, n_estimators=80, score=-3.393, total=   0.2s
[CV] max_depth=7, min_samples_leaf=3, n_estimators=80 ................
[CV]  max_depth=7, min_samples_leaf=3, n_estimators=80, score=0.255, total=   0.2s
[CV] max_depth=7, min_samples_leaf=3, n_estimators=80 ................
[CV]  max_depth=7, min_samples_leaf=3, n_estimators=80, score=0.377, total=   0.2s
[CV] max_depth=7, min_samples_leaf=3, n_estimators=80 ................
[CV]  max_depth=7, min_samples_leaf=3, n_estimators=80, score=0.247, total=   0.2s
[CV] max_depth=7, min_samples_leaf=3, n_estimators=80 ................
[CV]  max_depth=7, min_samples_leaf=3, n_estimators=80, score=0.189, total=

...

[CV]  max_depth=9, min_samples_leaf=4, n_estimators=200, score=0.226, total=   0.5s
[CV] max_depth=9, min_samples_leaf=4, n_estimators=200 ...............
[CV]  max_depth=9, min_samples_leaf=4, n_estimators=200, score=0.165, total=   0.7s
[CV] max_depth=9, min_samples_leaf=4, n_estimators=400 ...............
[CV]  max_depth=9, min_samples_leaf=4, n_estimators=400, score=-3.527, total=   1.4s
[CV] max_depth=9, min_samples_leaf=4, n_estimators=400 ...............
[CV]  max_depth=9, min_samples_leaf=4, n_estimators=400, score=0.312, total=   0.9s
[CV] max_depth=9, min_samples_leaf=4, n_estimators=400 ...............
[CV]  max_depth=9, min_samples_leaf=4, n_estimators=400, score=0.379, total=   1.0s
[CV] max_depth=9, min_samples_leaf=4, n_estimators=400 ...............
[CV]  max_depth=9, min_samples_leaf=4, n_estimators=400, score=0.225, total=   0.8s
[CV] max_depth=9, min_samples_leaf=4, n_estimators=400 ...............
[CV]  max_depth=9, min_samples_leaf=4, n_estimators=400, score=0.191,

[CV]  max_depth=10, min_samples_leaf=3, n_estimators=80, score=0.348, total=   0.2s
[CV] max_depth=10, min_samples_leaf=3, n_estimators=80 ...............
[CV]  max_depth=10, min_samples_leaf=3, n_estimators=80, score=0.235, total=   0.2s
[CV] max_depth=10, min_samples_leaf=3, n_estimators=80 ...............
[CV]  max_depth=10, min_samples_leaf=3, n_estimators=80, score=0.230, total=   0.2s
[CV] max_depth=10, min_samples_leaf=3, n_estimators=90 ...............
[CV]  max_depth=10, min_samples_leaf=3, n_estimators=90, score=-3.537, total=   0.2s
[CV] max_depth=10, min_samples_leaf=3, n_estimators=90 ...............
[CV]  max_depth=10, min_samples_leaf=3, n_estimators=90, score=0.314, total=   0.2s
[CV] max_depth=10, min_samples_leaf=3, n_estimators=90 ...............
[CV]  max_depth=10, min_samples_leaf=3, n_estimators=90, score=0.390, total=   0.2s
[CV] max_depth=10, min_samples_leaf=3, n_estimators=90 ...............
[CV]  max_depth=10, min_samples_leaf=3, n_estimators=90, score=0.216,

[CV]  max_depth=10, min_samples_leaf=4, n_estimators=400, score=-3.269, total=   0.7s
[CV] max_depth=10, min_samples_leaf=4, n_estimators=400 ..............
[CV]  max_depth=10, min_samples_leaf=4, n_estimators=400, score=0.315, total=   0.7s
[CV] max_depth=10, min_samples_leaf=4, n_estimators=400 ..............
[CV]  max_depth=10, min_samples_leaf=4, n_estimators=400, score=0.392, total=   0.7s
[CV] max_depth=10, min_samples_leaf=4, n_estimators=400 ..............
[CV]  max_depth=10, min_samples_leaf=4, n_estimators=400, score=0.225, total=   0.7s
[CV] max_depth=10, min_samples_leaf=4, n_estimators=400 ..............
[CV]  max_depth=10, min_samples_leaf=4, n_estimators=400, score=0.201, total=   0.8s
[CV] max_depth=10, min_samples_leaf=4, n_estimators=600 ..............
[CV]  max_depth=10, min_samples_leaf=4, n_estimators=600, score=-3.583, total=   1.7s
[CV] max_depth=10, min_samples_leaf=4, n_estimators=600 ..............
[CV]  max_depth=10, min_samples_leaf=4, n_estimators=600, scor

[Parallel(n_jobs=1)]: Done 420 out of 420 | elapsed:  3.4min finished


GridSearchCV(cv=5, error_score='raise-deprecating',
             estimator=RandomForestRegressor(bootstrap=True, criterion='mse',
                                             max_depth=None,
                                             max_features='auto',
                                             max_leaf_nodes=None,
                                             min_impurity_decrease=0.0,
                                             min_impurity_split=None,
                                             min_samples_leaf=1,
                                             min_samples_split=2,
                                             min_weight_fraction_leaf=0.0,
                                             n_estimators='warn', n_jobs=None,
                                             oob_score=False, random_state=None,
                                             verbose=0, warm_start=False),
             iid='warn', n_jobs=None,
             param_grid={'max_depth': (7, 8, 9, 10),
   

In-sample score

In [56]:
y_pred_train = grid_.best_estimator_.predict(X_train_iq)
score = mean_absolute_error(y_train_iq, y_pred_train)
print(score)

2.1786146607794157


In [57]:
train_pred_iq['RF'] = y_pred_train

Out-of-sample score

In [58]:
y_pred_val_iq = calculate_validation_score(grid_.best_estimator_ ,X_val_iq.drop('city', axis=1))
score = mean_absolute_error(y_val_iq, [x for x in y_pred_val_iq[0]])
print(score)

10.070967741935483


In [59]:
val_result_iq['RF'] = y_pred_val_iq[0]

In [89]:
train_pred_sj.to_csv('approach2_insample_prediction_sj.csv', index=False)

In [90]:
train_pred_iq.to_csv('approach2_insample_prediction_iq.csv', index=False)

In [91]:
val_result_iq.to_csv('approach2_outsample_prediction_iq.csv', index=False)

In [92]:
val_result_sj.to_csv('approach2_outsample_prediction_sj.csv', index=False)